In [1]:
import cv2
import pandas
from datetime import datetime

In [2]:
first_frame = None
status_list=[None,None]
times=[]
df = pandas.DataFrame(columns=["Start","End"]) # Creating a df to store timestamps

video = cv2.VideoCapture(0)  # This activates your camera

while True:
    check,frame = video.read()
    status = 0

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # Convert every frame to grayscale
    gray = cv2.GaussianBlur(gray,(21,21),0) # Gaussian Blur

    if first_frame is None:
        first_frame=gray # storing the first frame
        continue

    # ------------------------------------------------------------------------

    # Creating other frames
    delta_frame = cv2.absdiff(first_frame, gray)  # Generating difference between first frame and rest of frames

    thresh_frame = cv2.threshold(delta_frame,30,255,cv2.THRESH_BINARY)[1] # Creating the black and white frame
    thresh_frame = cv2.dilate(thresh_frame,None,iterations=2)

    (cnts,_) = cv2.findContours(thresh_frame.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE) # This is to detect any objects

    # ------------------------------------------------------------------------

    # For drawing rectangles on different objects
    for contour in cnts:
        if cv2.contourArea(contour)<10000: # If an object area is less than 10,000 pixels...then ignore it
            continue

        status = 1

        (x,y,w,h)=cv2.boundingRect(contour)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),3) # Draw rectangles

        # To note the timestamp when an object is detected & when it is not
        status_list.append(status)
        if status_list[-1]==1 and status_list[-2]==0:
            times.append(datetime.now())

        if status_list[-1] == 0 and status_list[-2] == 1:
            times.append(datetime.now())

    # ------------------------------------------------------------------------

    cv2.imshow("GrayFrame", gray)  # Displays grey image
    cv2.imshow("DeltaFrame", delta_frame)  # Displays delta image
    cv2.imshow("ThresholdFrame", thresh_frame)  # Displays delta image

    key = cv2.waitKey(1)  # 1000 is 1 sec

    if key == ord('q'):  # If we press 'q' it will stop
        if status==1:
            times.append(datetime.now())
        break

# Creating the df
for i in range(0,len(times),2):
    df = df.append({"Start":times[i],"End":times[i+1]},ignore_index=True)

df.to_csv("Timestamp.csv")

video.release()
cv2.destroyAllWindows


IndexError: list index out of range

: 